# TSTR CTGAN Dataset E

In [2]:
#import libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import os
print('Libraries imported!!')

Libraries imported!!


In [3]:
#define directory of functions and actual directory
HOME_PATH = '' #home path of the project
FUNCTIONS_DIR = 'EVALUATION FUNCTIONS/UTILITY'
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(HOME_PATH + FUNCTIONS_DIR)

#import functions for data labelling analisys
from utility_evaluation import DataPreProcessor
from utility_evaluation import train_evaluate_model

#change directory to actual directory
os.chdir(ACTUAL_DIR)
print('Functions imported!!')

Functions imported!!


## 1. Read data

In [4]:
#read real dataset
train_data = pd.read_csv(HOME_PATH + 'SYNTHETIC DATASETS/CTGAN/E_PimaIndiansDiabetes_Synthetic_CTGAN.csv')
categorical_columns = ['Outcome']
for col in categorical_columns :
    train_data[col] = train_data[col].astype('category')
train_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,80,102,14,2,29.0,1.022,57,1
1,8,142,42,35,-16,42.8,0.114,23,1
2,1,116,66,12,-3,17.2,0.120,21,0
3,0,73,92,-2,39,22.8,0.293,56,1
4,0,213,67,2,268,24.2,0.503,29,1
...,...,...,...,...,...,...,...,...,...
609,5,130,73,11,586,36.5,0.562,55,1
610,0,139,73,9,-2,21.7,0.287,29,1
611,-1,77,57,29,-10,27.3,0.047,51,0
612,0,104,50,47,-11,36.6,0.847,59,1


In [5]:
#read test data
test_data = pd.read_csv(HOME_PATH + 'REAL DATASETS/TEST DATASETS/E_PimaIndiansDiabetes_Real_Test.csv')
for col in categorical_columns :
    test_data[col] = test_data[col].astype('category')
test_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98,58,33,190,34.0,0.430,43,0
1,2,112,75,32,0,35.7,0.148,21,0
2,2,108,64,0,0,30.8,0.158,21,0
3,8,107,80,0,0,24.6,0.856,34,0
4,7,136,90,0,0,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...,...
149,9,165,88,0,0,30.4,0.302,49,1
150,1,77,56,30,56,33.3,1.251,24,0
151,8,95,72,0,0,36.8,0.485,57,0
152,2,146,70,38,360,28.0,0.337,29,1


In [6]:
target = 'Outcome'
#quick look at the breakdown of class values
print('Train data')
print(train_data.shape)
print(train_data.groupby(target).size())
print('#####################################')
print('Test data')
print(test_data.shape)
print(test_data.groupby(target).size())

Train data
(614, 9)
Outcome
0    211
1    403
dtype: int64
#####################################
Test data
(154, 9)
Outcome
0    99
1    55
dtype: int64


## 2. Pre-process training data

In [7]:
target = 'Outcome'
categorical_columns = None
numerical_columns = train_data.select_dtypes(include=['int64','float64']).columns.tolist()
categories = None

data_preprocessor = DataPreProcessor(categorical_columns, numerical_columns, categories)
x_train = data_preprocessor.preprocess_train_data(train_data.loc[:, train_data.columns != target])
y_train = train_data.loc[:, target]

x_train.shape, y_train.shape

((614, 8), (614,))

## 3. Preprocess test data

In [8]:
x_test = data_preprocessor.preprocess_test_data(test_data.loc[:, test_data.columns != target])
y_test = test_data.loc[:, target]
x_test.shape, y_test.shape

((154, 8), (154,))

## 4. Create a dataset to save the results

In [9]:
results = pd.DataFrame(columns = ['model','accuracy','precision','recall','f1'])
results

,model,accuracy,precision,recall,f1


## 4. Train and evaluate Random Forest Classifier

In [10]:
rf_results = train_evaluate_model('RF', x_train, y_train, x_test, y_test)
results = results.append(rf_results, ignore_index=True)
rf_results

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished


,model,accuracy,precision,recall,f1
0,RF,0.4481,0.7106,0.4481,0.374


## 5. Train and Evaluate KNeighbors Classifier

In [11]:
knn_results = train_evaluate_model('KNN', x_train, y_train, x_test, y_test)
results = results.append(knn_results, ignore_index=True)
knn_results

,model,accuracy,precision,recall,f1
0,KNN,0.474,0.6214,0.474,0.4471


## 6. Train and evaluate Decision Tree Classifier

In [12]:
dt_results = train_evaluate_model('DT', x_train, y_train, x_test, y_test)
results = results.append(dt_results, ignore_index=True)
dt_results

,model,accuracy,precision,recall,f1
0,DT,0.4351,0.5737,0.4351,0.3975


## 7. Train and evaluate Support Vector Machines Classifier

In [13]:
svm_results = train_evaluate_model('SVM', x_train, y_train, x_test, y_test)
results = results.append(svm_results, ignore_index=True)
svm_results

[LibSVM]

,model,accuracy,precision,recall,f1
0,SVM,0.2662,0.3038,0.2662,0.2813


## 8. Train and evaluate Multilayer Perceptron Classifier

In [14]:
mlp_results = train_evaluate_model('MLP', x_train, y_train, x_test, y_test)
results = results.append(mlp_results, ignore_index=True)
mlp_results

Iteration 1, loss = 0.66525092
Iteration 2, loss = 0.62842573
Iteration 3, loss = 0.61250394
Iteration 4, loss = 0.60457144
Iteration 5, loss = 0.60035791
Iteration 6, loss = 0.59529669
Iteration 7, loss = 0.59059339
Iteration 8, loss = 0.58577982
Iteration 9, loss = 0.58251283
Iteration 10, loss = 0.57906775
Iteration 11, loss = 0.57642992
Iteration 12, loss = 0.57384669
Iteration 13, loss = 0.57114402
Iteration 14, loss = 0.56870136
Iteration 15, loss = 0.56530111
Iteration 16, loss = 0.56197226
Iteration 17, loss = 0.55872843
Iteration 18, loss = 0.55654354
Iteration 19, loss = 0.55367427
Iteration 20, loss = 0.55183118
Iteration 21, loss = 0.54857647
Iteration 22, loss = 0.54573532
Iteration 23, loss = 0.54285251
Iteration 24, loss = 0.54050451
Iteration 25, loss = 0.53826577
Iteration 26, loss = 0.53553356
Iteration 27, loss = 0.53413514
Iteration 28, loss = 0.53474493
Iteration 29, loss = 0.53066609
Iteration 30, loss = 0.52550152
Iteration 31, loss = 0.52302501
Iteration 32, los

,model,accuracy,precision,recall,f1
0,MLP,0.4935,0.6052,0.4935,0.4846


## 9. Save results file

In [15]:
results.to_csv('RESULTS/models_results_ctgan.csv', index=False)
results

,model,accuracy,precision,recall,f1
0,RF,0.4481,0.7106,0.4481,0.3740
1,KNN,0.4740,0.6214,0.4740,0.4471
2,DT,0.4351,0.5737,0.4351,0.3975
3,SVM,0.2662,0.3038,0.2662,0.2813
4,MLP,0.4935,0.6052,0.4935,0.4846
